This notebook tries to train on the finalised dataset (196GB).

In [4]:
import pandas as pd

df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
#df["sentence"] = df.norm_words.apply(" ".join)
df = df.rename(columns={"path":"hashname"})

def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)
df["sentence"] = df.words.apply(" ".join).apply(process)


In [5]:
df[df.hashname.isin(["4_yZqDEYbaw_11856.34-11870.3.wav", "DUsdmFyClVI_2296.14-2315.9.wav"])]

,hashname,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,speaker_info,split,norm_words_edited,sentence
915,DUsdmFyClVI_2296.14-2315.9.wav,"12 4 2019 - 1. dio, 11. sjednica, 9. saziv [DU...",2296.14,2315.9,"[više, nije, održiva, dugoročno, i, mislim, da...","[0, 0.41000000000000003, 1.33, 1.9100000000000...","[više, nije, održiva, dugoročno, i, mislim, da...","[0, 0.41000000000000003, 1.33, 1.9100000000000...",ParlaMint-HR_S11.u10999,ParlaMint-HR_S11.u10999,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",train,NaN,više nije održiva dugoročno i mislim da će se ...
235024,4_yZqDEYbaw_11856.34-11870.3.wav,"20 6 2018 - 8. sjednica, 9. saziv [4_yZqDEYbaw...",11856.34,11870.3,"[je, regulirana, problematika, obavljanja,, or...","[0, 0.47000000000000003, 1.21, 1.92, 2.46, 3.1...","[je, regulirana, problematika, obavljanja, org...","[0, 0.47000000000000003, 1.21, 1.92, 2.46, 3.1...",ParlaMint-HR_S08.u8101,ParlaMint-HR_S08.u8101,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",train,NaN,je regulirana problematika obavljanja organizi...


In [3]:
df = pd.concat([df, df.speaker_info.apply(pd.Series)], axis=1)

In [4]:
test_df = df[df.split=="dev"].copy()
is_train = df.split=="train" 
has_speaker_name =~df.Speaker_name.isna()
speaker_name_is_not_dash = df.Speaker_name != "-"
train_df = df[is_train & has_speaker_name & speaker_name_is_not_dash].copy()


In [6]:
troubled_idx = [170,
 474,
 778,
 1082,
 1385,
 1688,
 1991,
 2294,
 2597,
 2900,
 3203,
 3506,
 3809,
 4111,
 4413,
 4715,
 5017,
 5319,
 5620,
 5921,
 6222,
 6523,
 6823,
 7123,
 7422,
 7721,
 8020,
 8319,
 8618,
 8917,
 9216,
 9515,
 9814,
 10113,
 10412,
 10711,
 11010,
 11309,
 11608,
 11907,
 12206,
 12505,
 12804,
 13103,
 13401,
 13699,
 13997,
 14295,
 14593,
 14891,
 15189,
 15487,
 15785,
 16083,
 16381,
 16679,
 16977,
 17275,
 17573,
 17871,
 18169,
 18467,
 18765,
 19063,
 19361,
 19659,
 19957,
 20255,
 20553,
 20850,
 21147,
 21444,
 21741,
 22038,
 22335,
 22632,
 22929,
 23226,
 23523,
 23820,
 24117,
 24414,
 24711,
 25008,
 25305,
 25602,
 25899,
 26196,
 26493,
 26790,
 27087,
 27384,
 27681,
 27978,
 28275,
 28572,
 28869,
 29166,
 29463,
 29760,
 30057,
 30354,
 30651,
 30948,
 31245,
 31542,
 31839,
 32136,
 32433,
 32730,
 33027,
 33324,
 33621,
 33918,
 34215,
 34512,
 34809,
 35106,
 35402,
 35698,
 35994,
 36290,
 36586,
 36882,
 37178,
 37474,
 37770,
 38066,
 38362,
 38658,
 38953,
 39248,
 39543,
 39838,
 40133,
 40428,
 40723,
 41018,
 41313,
 41608,
 41903,
 42198,
 42493,
 42788,
 43083,
 43378,
 43673,
 43968,
 44263,
 44558,
 44853,
 45148,
 45443,
 45738,
 46033,
 46328,
 46623,
 46918,
 47213,
 47508,
 47803,
 48098,
 48393,
 48688,
 48983,
 49278,
 49573,
 49868,
 50163,
 50458,
 50753,
 51048,
 51343,
 51638,
 51933,
 52228,
 52523,
 52818,
 53113,
 53408,
 53703,
 53998,
 54293,
 54588,
 54883,
 55178,
 55473,
 55768,
 56063,
 56358,
 56653,
 56948,
 57242,
 57536,
 57830,
 58124,
 58418,
 58712,
 59006,
 59300,
 59594,
 59888,
 60182,
 60476,
 60770,
 61064,
 61358,
 61652,
 61946,
 62240,
 62534,
 62828,
 63122,
 63416,
 63710,
 64004,
 64298,
 64592,
 64885,
 65178,
 65471,
 65764,
 66057,
 66350,
 66643,
 66936,
 67229,
 67522,
 67815,
 68108,
 68401,
 68694,
 68987,
 69280,
 69573,
 69866,
 70159,
 70452,
 70745,
 71037,
 71329,
 71621,
 71913,
 72205,
 72497,
 72789,
 73081,
 73373,
 73665,
 73957,
 74249,
 74541,
 74833,
 75125,
 75417,
 75709,
 76001,
 76293,
 76585,
 76877,
 77169,
 77461,
 77753,
 78045,
 78337,
 78629,
 78921,
 79213,
 79505,
 79797,
 80089,
 80381,
 80673,
 80965,
 81257,
 81549,
 81840,
 82131,
 82422,
 82713,
 83004,
 83295,
 83586,
 83877,
 84168,
 84459,
 84750,
 85041,
 85332,
 85623,
 85914,
 86205,
 86496,
 86787,
 87078,
 87369,
 87660,
 87951,
 88242,
 88533,
 88824,
 89115,
 89406,
 89697,
 89988,
 90279,
 90570,
 90861,
 91152,
 91443,
 91734,
 92025,
 92316,
 92607,
 92898,
 93189,
 93480,
 93771,
 94062,
 94353,
 94644,
 94935,
 95226,
 95516,
 95806,
 96096,
 96386,
 96676,
 96966,
 97255,
 97544,
 97833,
 98122,
 98411,
 98700,
 98989,
 99278,
 99567,
 99856,
 100145,
 100434,
 100723,
 101012,
 101301,
 101590,
 101879,
 102168,
 102457,
 102746,
 103035,
 103324,
 103613,
 103902,
 104191,
 104480,
 104769,
 105058,
 105347,
 105635,
 105923,
 106211,
 106499,
 106787,
 107075,
 107363,
 107651,
 107939,
 108227,
 108514,
 108801,
 109088,
 109375,
 109662,
 109949,
 110236,
 110523,
 110810,
 111097,
 111384,
 111671,
 111958,
 112245,
 112532,
 112819,
 113106,
 113393,
 113680,
 113967,
 114254,
 114541,
 114828,
 115115,
 115402,
 115689,
 115976,
 116263,
 116550,
 116836,
 117122,
 117408,
 117694,
 117980,
 118266,
 118551,
 118836,
 119121,
 119406,
 119691,
 119976,
 120261,
 120546,
 120831,
 121116,
 121401,
 121686,
 121971,
 122256,
 122541,
 122826,
 123111,
 123396,
 123681,
 123966,
 124251,
 124536,
 124821,
 125106,
 125391,
 125676,
 125961,
 126246,
 126531,
 126816,
 127101,
 127386,
 127671,
 127956,
 128241,
 128526,
 128811,
 129096,
 129380,
 129664,
 129948,
 130232,
 130516,
 130800,
 131084,
 131368,
 131652,
 131936,
 132220,
 132504,
 132788,
 133072,
 133356,
 133640,
 133924,
 134208,
 134492,
 134776,
 135060,
 135344,
 135628,
 135912,
 136196,
 136480,
 136764,
 137048,
 137332,
 137616,
 137900,
 138184,
 138468,
 138752,
 139036,
 139320,
 139604,
 139888,
 140172,
 140456,
 140740,
 141024,
 141308,
 141592,
 141876,
 142160,
 142444,
 142728,
 143012,
 143296,
 143579,
 143862,
 144145,
 144428,
 144711,
 144994,
 145277,
 145560,
 145843,
 146126,
 146409,
 146692,
 146975,
 147258,
 147541,
 147824,
 148107,
 148390,
 148673,
 148956,
 149239,
 149522,
 149805,
 150088,
 150371,
 150654,
 150937,
 151220,
 151502,
 151784,
 152066,
 152348,
 152630,
 152912,
 153194,
 153476,
 153758,
 154040,
 154322,
 154604,
 154886,
 155168,
 155450,
 155732,
 156014,
 156296,
 156578,
 156860,
 157142,
 157424,
 157706,
 157988,
 158270,
 158552,
 158834,
 159116,
 159398,
 159680,
 159962,
 160244,
 160526,
 160808,
 161090,
 161372,
 161654,
 161936,
 162218,
 162500,
 162782,
 163064,
 163346,
 163628,
 163910,
 164192,
 164474,
 164756,
 165038,
 165320,
 165602,
 165884,
 166166,
 166448,
 166730,
 167012,
 167294,
 167576,
 167858,
 168140,
 168422,
 168704,
 168986,
 169268,
 169550,
 169832,
 170114,
 170396,
 170678,
 170960,
 171242,
 171524,
 171806,
 172088,
 172370,
 172652,
 172934,
 173216,
 173498,
 173780,
 174062,
 174344,
 174626,
 174908,
 175190,
 175472,
 175754,
 176036,
 176318,
 176600,
 176882,
 177164,
 177446,
 177728,
 178009,
 178290,
 178571,
 178852,
 179133,
 179414,
 179695,
 179976,
 180257,
 180538,
 180819,
 181100,
 181381,
 181662,
 181943,
 182224,
 182505,
 182786,
 183067,
 183348,
 183629,
 183910,
 184191,
 184472,
 184753,
 185034,
 185315,
 185596,
 185877,
 186158,
 186439,
 186720,
 187001,
 187282,
 187563,
 187844,
 188125,
 188406,
 188687,
 188968,
 189249,
 189530,
 189811,
 190092,
 190373,
 190654,
 190935,
 191216,
 191497,
 191777,
 192057,
 192337,
 192617,
 192897,
 193177,
 193457,
 193737,
 194017,
 194297,
 194577,
 194857,
 195137,
 195417,
 195697,
 195977,
 196257,
 196537,
 196817,
 197097,
 197377,
 197657,
 197937,
 198217,
 198497,
 198777,
 199057,
 199337,
 199617,
 199897,
 200177,
 200457,
 200737,
 201017,
 201297,
 201577,
 201857,
 202137,
 202417,
 202697,
 202977,
 203257,
 203537,
 203817,
 204097,
 204377,
 204657,
 204937,
 205217,
 205497,
 205777,
 206057,
 206337,
 206617,
 206897,
 207177,
 207457,
 207737,
 208017,
 208297,
 208577,
 208857,
 209137,
 209417,
 209697,
 209977,
 210257,
 210537,
 210817,
 211097,
 211377,
 211657,
 211937,
 212217,
 212497,
 212777,
 213057,
 213337,
 213617,
 213897,
 214177,
 214457,
 214737,
 215017,
 215296,
 215575,
 215854,
 216133,
 216412,
 216691,
 216970,
 217249,
 217528,
 217807,
 218086,
 218365,
 218644,
 218923,
 219202,
 219481,
 219760,
 220039]

for i in troubled_idx:
    hashname = train_df.iloc[i, 0]
    fullpath = f"/home/rupnik/macocu/task8/data_16000_mono/seg.{hashname}"
    if not os.path.exists(fullpath):
        print(f"path {fullpath} does not exist!")
    from pydub import AudioSegment
    audio = AudioSegment.from_wav(fullpath)
    if len(audio) == 0:
        print(f"path {path} has length 0!")

In [7]:
len(troubled_idx)

764

Heavy lifting: the cell below takes a long while to run due to many edits to the train_df dataframe


In [8]:
new_indices = []
target_size = 220000
while len(new_indices) < target_size:
    speakers = train_df.Speaker_name.unique()
    for speaker in speakers:
        ind = train_df[train_df.Speaker_name == speaker].index[0]
        new_indices.append(ind)
    train_df = train_df.drop(index=ind)
    print(f"{len(new_indices)=}, {len(speakers)=}", end = "\r")

train_df = df[is_train & has_speaker_name & speaker_name_is_not_dash].copy()
train_df = train_df.loc[new_indices, :]

In [14]:
new_indices

[0,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 59,
 66,
 68,
 73,
 74,
 75,
 78,
 80,
 86,
 91,
 92,
 93,
 94,
 95,
 99,
 100,
 105,
 109,
 119,
 120,
 124,
 126,
 129,
 131,
 135,
 146,
 150,
 152,
 163,
 165,
 171,
 178,
 182,
 183,
 197,
 202,
 203,
 209,
 211,
 212,
 213,
 217,
 220,
 223,
 224,
 226,
 227,
 232,
 236,
 242,
 243,
 249,
 250,
 261,
 265,
 267,
 268,
 272,
 280,
 281,
 282,
 289,
 290,
 302,
 307,
 310,
 311,
 312,
 314,
 316,
 322,
 334,
 341,
 342,
 349,
 351,
 353,
 357,
 366,
 370,
 394,
 399,
 409,
 411,
 430,
 434,
 447,
 451,
 452,
 471,
 473,
 487,
 492,
 500,
 511,
 513,
 518,
 519,
 526,
 548,
 549,
 555,
 562,
 566,
 569,
 574,
 575,
 614,
 617,
 623,
 646,
 653,
 673,
 676,
 705,
 706,
 713,
 740,
 742,
 777,
 779,
 784,
 818,
 882,
 887,
 891,
 900,
 913,
 915,
 957,
 976,
 995,
 997,
 1037,
 1080,


In [28]:
train_df = df[is_train & has_speaker_name & speaker_name_is_not_dash].copy()
train_df = train_df.loc[new_indices, :]
test_df = df[df.split=="test"].copy()

In [30]:
def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)
train_df["sentence"] = train_df["words"].apply(" ".join).apply(process)
test_df["sentence"] = test_df["words"].apply(" ".join).apply(process)

In [31]:
columns_to_keep = ["hashname", "sentence"]
train_df = train_df.loc[:, columns_to_keep]
test_df = test_df.loc[:, columns_to_keep]


In [32]:
import os
# Use old or new vocab?
os.system("cp vocab_300_with_numbers.json vocab.json")


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:

import gc
gc.collect()

train_df["audio"] = train_df.hashname.apply(lambda s: "data_16000_mono/seg."+s)
test_df["audio"] = test_df.hashname.apply(lambda s: "data_16000_mono/seg."+s)


import datasets
from datasets import load_dataset, load_metric, Audio

train_dataset = datasets.Dataset.from_pandas(train_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

train_dataset = train_dataset.cast_column("audio", Audio())
test_dataset = test_dataset.cast_column("audio", Audio())

del train_df
del test_df
gc.collect()

35

In [34]:

from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

train_mapped = train_dataset.map(
    prepare_dataset, remove_columns=train_dataset.column_names, 
    #num_proc=8, 
    cache_file_name=".cache_train", )
test_mapped = test_dataset.map(
    prepare_dataset, remove_columns=test_dataset.column_names, 
    #num_proc=8, 
    cache_file_name=".cache_test", )
print("Data Preparation Complete!")

del train_dataset
del test_dataset

gc.collect()

  0%|          | 0/220148 [00:00<?, ?ex/s]

  0%|          | 0/513 [00:00<?, ?ex/s]

Data Preparation Complete!


2827

In [35]:
train_mapped.save_to_disk("train220k_mapped")

In [36]:
test_mapped.save_to_disk("test_mapped")

In [37]:
!rsync -e "ssh" -av ~/macocu/task8/{test,train220k}_mapped virtualka:/home/peterr/macucu/task8/

Enter passphrase for key '/home/rupnik/.ssh/id_rsa': 
rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at rsync.c(644) [sender=3.1.3]
